# **MIP4_Run2**

In [2]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
glpk-utils is already the newest version (5.0-1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
  Using cached glpk-0.4.7.tar.gz (161 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for glpk (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for glpk
Failed to build glpk
ERROR: Could not build wheels for glpk, which is required to install pyproject.toml-based projects


In [3]:

import openai
import os
from IPython.display import Markdown


### **Accessing the GPT4 API**

In [4]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']
openai.api_key = my_api_key


### **Generate Mathematical Model**

In [5]:
problem = """You are task with maximizing the water flow in a network of pipes over 10 time steps. The water enters the system at point A and exits the system at point G. Hence, you need to maximize the amount of water that flows to point G. The water enters point A according to the function f(t) = max(-4/30t^3+t^2-0.234t+3, 0), where t is the time step. There are serveral points that are connected through pipes. Each pipe has a fixed capacity. Each point has to possibility to release water out of the system in case the outgoing pipes are full. There are 12 available pipes in total but only 10 of them can be activated. Deactivated pipes can not be used to transport water. The activation status of a pipe is set before any water arrives at point A and does not change. Finally, water can not be stored at the connection points which means that any water coming in at time t needs to be flowing out at time t + 1.

These are the pipe capacities:
AB: 3
AC: 6
AF: 1
BC: 3
BD: 10
BE: 4
CE: 4
CF: 4
DE: 5
EG: 5
FB: 12
FG: 7"""

In [6]:

client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the variables for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 2
)


In [7]:
print(response1.choices[0].message.content)

1. Decision Variables:
   - xi,j ∈ {0,1} for all i,j ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG} (1 if pipe i,j is activated and 0 if not activated)
   - yi,j,t  for all i,j ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG} and t = {1..10}; (amount of water flowing through pipe i,j at time t)

2. Parameters:
   - Capacityj ∈ {3, 6, 1, 3, 10, 4, 4, 4, 5, 5, 12, 7} for every pipe j ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}
   - InputWater_A_t = max(-4/30*t^3+t^2-0.234*t+3, 0) for every time step t
   
Note: The subscript i,j represents the pipe from point i to point j. For instance, AB represents the pipe from point A to point B. Similarly, subscript t represents the time step.


In [8]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the objective function for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [9]:
print(response2.choices[0].message.content)

Objective Function:

Maximize sum(t=1 to 10) y_EG,t
subject to:
For each i,j ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG} and each t ∈ {1..10}:
  - xi,j * y_i,j,t ≤ Capacityj * xi,j 
where,
xi,j denotes whether pipe i,j is activated, and y_i,j,t denotes the amount of water flowing through pipe i,j at time t.


In [10]:
response3 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate only the constraints for this mathematical optimization problem."},
        {
            "role": "user",
            "content": problem + response1.choices[0].message.content +  response2.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [11]:
print(response3.choices[0].message.content)

1. Total pipes activated constraint:
Sum(xi,j for all i,j ∈ {AB, AC, AF, BC, BD, BE, CE, CF, DE, EG, FB, FG}) <= 10

where xi,j denotes whether pipe i,j is activated.

2. Flow conservation constraint for each point in each timestep t ∈ {1,2,3,...,10}
Sum(yi,j,t for all outgoing j from each point) = Sum(yj,i,t+1 for all incoming i to that point)

where yi,j,t denotes the amount of water flowing through pipe i,j at time t.

3. Capacity constraints for each pipe in each timestep t ∈ {1,2,3,...,10}
yi,j,t <= xi,j * Capacityj 

where yi,j,t denotes the amount of water flowing through pipe i,j at time t and xi,j denotes whether pipe i,j is activated.

4. Water can't be stored at the connection points constraint:
Sum(yj,i,t for all incoming i to that point) <= Sum(yi,j,t for all outgoing j from each point)

where yi,j,t denotes the amount of water flowing through pipe i,j at time t.

5. The water flow into the system can be represented by:
yi,A,t <= InputWater_A_t

where yi,A,t denotes the am

### **Generate Pyomo Code**

In [20]:
response4 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content + response2.choices[0].message.content + response3.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 2
)

In [21]:
Markdown(response4.choices[0].message.content)

Here is a Python code using pyomo for the given problem assuming the InputWater_A_t as the constraint:

```python
import numpy as np
from pyomo.environ import *

#change your set according to the problem
set_ = ['AB', 'AC', 'AF', 'BC', 'BD', 'BE', 'CE', 'CF', 'DE', 'EG', 'FB', 'FG']
pipe_capacity = {'AB':3, 'AC':6, 'AF':1, 'BC':3, 'BD':10, 'BE':4, 'CE':4, 'CF':4, 'DE':5, 'EG':5, 'FB':12, 'FG':7}

model = ConcreteModel()

# Decision Variables
model.x = Var(set_, within=Binary)
model.y = Var(set_, within=NonNegativeReals)

def Obj_rule(model):
    return sum(model.y[i] for i in set_)
model.Obj = Objective(rule=Obj_rule, sense=maximize)

def capacity_constraints(model, i):
        return model.x[i]*model.y[i] <= pipe_capacity[i]*model.x[i]
model.capacity_cons = Constraint(set_, rule=capacity_constraints)

def total_pipes_rule(model):
        return sum(model.x[i]  for i in set_) <= 10
model.total_pipes_cons = Constraint(rule=total_pipes_rule)

def flow_conservation_rule(model, i):
    return sum(model.x[j]*model.y[i] for j in set_) == sum(model.x[i]*model.y[j] for j in set_)
model.flow_conservation_cons = Constraint(set_, rule=flow_conservation_rule)

def water_storage_rule(model, i):
    return sum(model.x[j]*model.y[i] for j in set_) <= sum(model.y[j]*model.x[i] for j in set_)
model.water_storage_cons = Constraint(set_, rule=water_storage_rule)

def input_water_rule(model, i):
    return model.y[i] <= max(-4/30*value(model.y[i])**3+value(model.y[i])**2-0.234*value(model.y[i])+3, 0)
model.input_water_cons = Constraint(set_, rule=input_water_rule)

SolverFactory('glpk').solve(model)
model.pprint()
```
Please note that this is a simplified version for explanation purposes. You would need to define your sets, parameters, and constraints depending upon the specific needs of your problem.

### **Run the code Generated by GPT4**

In [23]:
import numpy as np
from pyomo.environ import *

# change your set according to the problem
set_ = ['AB', 'AC', 'AF', 'BC', 'BD', 'BE', 'CE', 'CF', 'DE', 'EG', 'FB', 'FG']
pipe_capacity = {'AB':3, 'AC':6, 'AF':1, 'BC':3, 'BD':10, 'BE':4, 'CE':4, 'CF':4, 'DE':5, 'EG':5, 'FB':12, 'FG':7}

model = ConcreteModel()

# Decision Variables
model.x = Var(set_, within=Binary)
model.y = Var(set_, within=NonNegativeReals)

def Obj_rule(model):
    return sum(model.y[i] for i in set_)
model.Obj = Objective(rule=Obj_rule, sense=maximize)

def capacity_constraints(model, i):
        return model.x[i]*model.y[i] <= pipe_capacity[i]*model.x[i]
model.capacity_cons = Constraint(set_, rule=capacity_constraints)

def total_pipes_rule(model):
        return sum(model.x[i]  for i in set_) <= 10
model.total_pipes_cons = Constraint(rule=total_pipes_rule)

def flow_conservation_rule(model, i):
    return sum(model.x[j]*model.y[i] for j in set_) == sum(model.x[i]*model.y[j] for j in set_)
model.flow_conservation_cons = Constraint(set_, rule=flow_conservation_rule)

def water_storage_rule(model, i):
    return sum(model.x[j]*model.y[i] for j in set_) <= sum(model.y[j]*model.x[i] for j in set_)
model.water_storage_cons = Constraint(set_, rule=water_storage_rule)

def input_water_rule(model, i):
    return model.y[i] <= max(-4/30*value(model.y[i])**3+value(model.y[i])**2-0.234*value(model.y[i])+3, 0)
model.input_water_cons = Constraint(set_, rule=input_water_rule)

SolverFactory('glpk').solve(model)
model.pprint()

ERROR:pyomo.common.numeric_types:evaluating object as numeric value: y[AB]
    (object: <class 'pyomo.core.base.var.VarData'>)
No value for uninitialized NumericValue object y[AB]
ERROR:pyomo.core:Rule failed when generating expression for Constraint input_water_cons with index AB:
ValueError: No value for uninitialized NumericValue object y[AB]
ERROR:pyomo.core:Constructing component 'input_water_cons' from data=None failed:
    ValueError: No value for uninitialized NumericValue object y[AB]


ValueError: No value for uninitialized NumericValue object y[AB]

### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**